In [ ]:
import pathlib
import hashlib
import json
import functools

import requests
import keyring

In [ ]:
ACCESS_TOKEN = keyring.get_password("Zenodo", "Token")
ACCESS_TOKEN

In [ ]:
def get_zenodo_access_token():
    access_token = keyring.get_password("Zenodo", "Token")

    if access_token is None or access_token == "":
        print(
            "To upload files to Zenodo you need to provide a Zenodo "
            "access token. Please go to "
            "<https://zenodo.org/account/settings/applications/tokens/new/>, "
            "login to Zenodo and create a new access token.\n"
            "When creating the access token use the scopes `deposit:actions`, `deposit:write`, "
            "and `user:email`. Once you have your token copy it into the prompt below "
            "to continue with the upload."
        )
        access_token = getpass()
        keyring.set_password("Zenodo", "Token", access_token)

    return access_token

In [ ]:
def upload_zenodo_file(filepath, use_sandbox=False, access_token=None):
    filepath = pathlib.Path(filepath)
    
    if use_sandbox:
        zenodo_url = 'https://sandbox.zenodo.org/'
    else:
        zenodo_url = 'https://zenodo.org/'
        
    if access_token is None:
        access_token = get_zenodo_access_token()
    
    depositions_url = f"{zenodo_url}api/deposit/depositions"
    
    headers = {"Content-Type": "application/json"}
    r = requests.post(
        depositions_url,
        params={'access_token': ACCESS_TOKEN}, json={},
        headers=headers)
    
    

In [ ]:
r = requests.get(
    'https://zenodo.org/api/deposit/depositions',
    params={'access_token': ACCESS_TOKEN})
r.json()

In [ ]:
headers = {"Content-Type": "application/json"}
r = requests.post('https://zenodo.org/api/deposit/depositions',
                  params={'access_token': ACCESS_TOKEN}, json={},
                  headers=headers)
r.json()

In [ ]:
deposition_id = r.json()['id']
filepath = pathlib.Path('test.txt')

In [ ]:
data = {'name': filepath.name}

with open(filepath, 'rb') as upload_file:
    files = {'file': upload_file}
    r = requests.post(f'https://zenodo.org/api/deposit/depositions/{deposition_id}/files',
                      params={'access_token': ACCESS_TOKEN}, data=data,
                      files=files)
r

In [ ]:
m = hashlib.md5()

with open(filepath, 'rb') as upload_file:
    m.update(upload_file.read())
    
md5 = m.hexdigest()
md5

In [ ]:
r.json()['checksum'] == md5

In [ ]:
def create_metadata(title, author='PyMedPhys Contributors'):
    metadata = {
        'metadata': {
            'title': title,
            'upload_type': 'dataset',
            'creators': [{'name': author}],
            'description': "<p>This is an automated upload from the PyMedPhys library.</p>",
            'license': 'Apache-2.0',
            'access_right': 'open',
        }
    }
    
    return json.dumps(metadata)


metadata = create_metadata('a_test', 'Biggs, Simon')
metadata

In [ ]:
r = requests.put(f'https://zenodo.org/api/deposit/depositions/{deposition_id}',
                 params={'access_token': ACCESS_TOKEN}, data=metadata,
                 headers=headers)

In [ ]:
r.json()